In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('deliveries.csv', encoding = "ISO-8859-1", na_filter=False)

In [3]:
df.head()

,season,match_id,venue,inning,batting_team,bowling_team,batsman,non_striker,bowler,ball,...,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,wickets,commentary
0,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0.1,...,0,0,0,1,1,,,,0,"Kumar to Ganguly, 1 leg bye, and thats runs im..."
1,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0.2,...,0,0,0,0,0,,,,0,"Kumar to McCullum, no run, this one nips back ..."
2,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0.3,...,0,0,0,1,1,,,,0,"Kumar to McCullum, 1 wide, thats wide, as Prav..."
3,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0.3,...,0,0,0,0,0,,,,0,"Kumar to McCullum, no run, too straight, McCul..."
4,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0.4,...,0,0,0,0,0,,,,0,"Kumar to McCullum, no run, steps down the trac..."


In [4]:
df.tail()

,season,match_id,venue,inning,batting_team,bowling_team,batsman,non_striker,bowler,ball,...,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,wickets,commentary
131996,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,RA Jadeja,SR Watson,SL Malinga,19.2,...,0,0,1,0,1,,,,5,"Malinga to Jadeja, 1 run, low full toss on mid..."
131997,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,SR Watson,RA Jadeja,SL Malinga,19.3,...,0,0,2,0,2,,,,5,"Malinga to Watson, 2 runs, yorker, just outsid..."
131998,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,SR Watson,RA Jadeja,SL Malinga,19.4,...,0,0,1,0,1,SR Watson,run out,KH Pandya,6,"Malinga to Watson, 1 run, OUT, yorker, just ou..."
131999,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,SN Thakur,RA Jadeja,SL Malinga,19.5,...,0,0,2,0,2,,,,6,"Malinga to Thakur, 2 runs, full toss on leg, a..."
132000,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,SN Thakur,RA Jadeja,SL Malinga,19.6,...,0,0,0,0,0,SN Thakur,lbw,,7,"Malinga to Thakur, OUT, Mumbai Indians win IPL..."


In [5]:
df.shape

(132001, 23)

In [6]:
df.columns

Index(['season', 'match_id', 'venue', 'inning', 'batting_team', 'bowling_team',
       'batsman', 'non_striker', 'bowler', 'ball', 'wide_runs', 'bye_runs',
       'legbye_runs', 'noball_runs', 'penalty_runs', 'batsman_runs',
       'extra_runs', 'total_runs', 'player_dismissed', 'dismissal_kind',
       'fielder', 'wickets', 'commentary'],
      dtype='object')

In [7]:
df.isna().sum()

season              0
match_id            0
venue               0
inning              0
batting_team        0
bowling_team        0
batsman             0
non_striker         0
bowler              0
ball                0
wide_runs           0
bye_runs            0
legbye_runs         0
noball_runs         0
penalty_runs        0
batsman_runs        0
extra_runs          0
total_runs          0
player_dismissed    0
dismissal_kind      0
fielder             0
wickets             0
commentary          0
dtype: int64

In [8]:
df.dtypes

season                int64
match_id             object
venue                object
inning                int64
batting_team         object
bowling_team         object
batsman              object
non_striker          object
bowler               object
ball                float64
wide_runs             int64
bye_runs              int64
legbye_runs           int64
noball_runs           int64
penalty_runs          int64
batsman_runs          int64
extra_runs            int64
total_runs            int64
player_dismissed     object
dismissal_kind       object
fielder              object
wickets               int64
commentary           object
dtype: object

In [9]:
df.batting_team.unique() 

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Mumbai Indians', 'Deccan Chargers',
       'Sunrisers Hyderabad', 'Delhi Capitals'], dtype=object)

In [10]:
columns_to_remove = ['batsman', 'non_striker', 'player_dismissed', 'fielder']
df.drop(labels=columns_to_remove, axis=1, inplace=True)

In [11]:
df.head()

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
0,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,P Kumar,0.1,0,0,1,0,0,0,1,1,,0,"Kumar to Ganguly, 1 leg bye, and thats runs im..."
1,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,P Kumar,0.2,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, this one nips back ..."
2,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,P Kumar,0.3,1,0,0,0,0,0,1,1,,0,"Kumar to McCullum, 1 wide, thats wide, as Prav..."
3,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,P Kumar,0.3,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, too straight, McCul..."
4,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,P Kumar,0.4,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, steps down the trac..."


In [12]:
df.loc[(df.batting_team=='Delhi Daredevils'),'batting_team'] = 'Delhi Capitals'
df.loc[(df.bowling_team=='Delhi Daredevils'),'bowling_team'] = 'Delhi Capitals'
df.loc[(df.batting_team=='Delhi Daredevils')|(df.bowling_team=='Delhi Daredevils')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary


In [13]:
rcb_spin = ['SB Joshi', 'KP Appanna', 'A Kumble', 'J Arunkumar', 'Abdur Razzak', 'CL White', 'TM Dilshan',
            'KP Pietersen', 'RE van der Merwe', 'S Sriram', 'DL Vettori', 'KB Arun Karthik', 'CH Gayle', 'S Sriram',
            'AA Kazi', 'R Ninan', 'J Syed Mohammad', 'M Muralitharan', 'M Kartik', 'P R Barman', 
            'YS Chahal', 'Yuvraj Singh', 'SB Jakati', 'Iqbal Abdulla', 'T Shamsi', 'Parvez Rasool', 
            'S Baby', 'TM Head', 'KM Jadhav', 'P Negi', 'S Badree', 'Washington Sundar', 'M Ali', 'M Ashwin']

mi_spin = ['A Dananjaya', 'AG Murtaza', 'A Roy', 'GJ Maxwell', 'Harbhajan Singh', 'J Suchith', 'J Yadav',
           'JP Duminy', 'KH Pandya', 'KV Sharma', 'M Markande', 'N Rana', 'PP Ojha', 'RD Chahar', 'RG Sharma',
           'RJ Peterson', 'RW Price', 'S Gopal', 'SD Chitnis', 'ST Jayasuriya', 'TL Suman', 'VS Yeligati']

kkr_spin = ['BAW Mendis','BJ Hodge', 'DJ Hussey', 'GB Hogg', 'J Botha', 'KC Cariappa', 'M Kartik', 'CH Gayle', 
            'Iqbal Abdulla', 'Kuldeep Yadav', 'MB Parmar', 'MK Tiwary', 'Mohammad Hafeez', 'N Rana', 
            'PP Chawla', 'RS Gavaskar', 'S Ladda', 'Shakib Al Hasan', 'SMSM Senanayake', 'SP Narine', 'YK Pathan']

csk_spin = ['DJ Hussey', 'F du Plessis', 'Harbhajan Singh', 'Imran Tahir', 'RA Jadeja', 'KV Sharma', 'M Santner', 
            'M Muralitharan', 'P Negi', 'R Ashwin', 'S Badree', 'S Randiv', 'S Vidyut', 'SB Jakati', 'SK Raina',
            'PP Chawla', 'RS Gavaskar', 'S Ladda', 'Shakib Al Hasan', 'SMSM Senanayake', 'SP Narine', 'YK Pathan']

dc_spin = ['A Mishra', 'AA Jhunjhunwala', 'Ankit Sharma', 'CL White', 'PP Ojha', 'DB Ravi Teja', 'JP Duminy',
           'LPC Silva', 'R Sharma', 'RG Sharma', 'S Dhawan', 'Shahid Afridi', 'TL Suman', 'Y Venugopal Rao']

dd_spin = ['A Mishra', 'AJ Finch', 'AR Patel', 'BMAJ Mendis', 'DL Vettori', 'GJ Maxwell', 'H Vihari', 'Imran Tahir',
           'J Suchith', 'J Botha', 'J Yadav', 'JP Duminy', 'KP Pietersen', 'MK Tiwary', 'P Negi', 'R Sharma', 
           'R Tewatia', 'RE van der Merwe', 'S Ladda', 'S Lamichhane', 'S Nadeem', 'S Sriram', 'Shoaib Malik', 
           'TM Dilshan', 'Sunny Gupta', 'V Sehwag', 'Y Venugopal Rao', 'Yuvraj Singh', 'Y Nagar']

srh_spin = ['A Mishra', 'Ankit Sharma', 'Bipul Sharma', 'CL White', 'DJ Hooda', 'KS Williamson', 'KV Sharma',
            'Mohammad Nabi', 'Parvez Rasool', 'Rashid Khan', 'S Nadeem', 'Shakib Al Hasan', 'Y Venugopal Rao', 'YK Pathan',
            'Yuvraj Singh']

rr_spin = ['A Chandila', 'AA Chavan', 'AA Jhunjhunwala', 'AC Voges', 'AJ Finch', 'AL Menaria', 'Ankit Sharma',
           'AS Raut', 'BJ Hodge', 'D Short', 'DJ Hooda', 'D Salunkhe', 'GB Hogg', 'I Sodhi', 'J Botha', 'K Gowtham',
           'L Livingstone', 'LRPL Taylor', 'M Lomror', 'PV Tambe', 'ND Doshi', 'R Parag', 'R Tewatia', 'S Badree',
           'S Gopal', 'RA Jadeja', 'S Midhun', 'SK Warne', 'YK Pathan']

punj_spin = ['AC Gilchrist', 'AR Patel', 'BA Bhatt', 'Bipul Sharma', 'DJ Hussey', 'GJ Maxwell', 'Gurkeerat Singh'
            'H Brar', 'Karanveer Singh', 'KC Cariappa', 'M Ashwin', 'M Kartik', 'M Ur Rahman', 'M Vijay', 'MK Tiwary',
             'P Sahu', 'PP Chawla', 'R Ashwin', 'R Tewatia', 'RR Powar', 'Shivam Sharma', 'SN Khan', 'Swapnil Singh',
             'V Chakravarthy', 'Yuvraj Singh']

In [14]:
df.loc[(df.bowling_team=='Royal Challengers Bangalore')&(df.bowler.isin(rcb_spin)),'bowler'] = 'spinner'
df.loc[(df.bowling_team=='Delhi Capitals')&(df.bowler.isin(dd_spin)),'bowler'] = 'spinner'
df.loc[(df.bowling_team=='Mumbai Indians')&(df.bowler.isin(mi_spin)),'bowler'] = 'spinner'
df.loc[(df.bowling_team=='Chennai Super Kings')&(df.bowler.isin(csk_spin)),'bowler'] = 'spinner'
df.loc[(df.bowling_team=='Kings XI Punjab')&(df.bowler.isin(punj_spin)),'bowler'] = 'spinner'
df.loc[(df.bowling_team=='Sunrisers Hyderabad')&(df.bowler.isin(srh_spin)),'bowler'] = 'spinner'
df.loc[(df.bowling_team=='Kolkata Knight Riders')&(df.bowler.isin(kkr_spin)),'bowler'] = 'spinner'
df.loc[(df.bowling_team=='Deccan Chargers')&(df.bowler.isin(dc_spin)),'bowler'] = 'spinner'
df.loc[(df.bowling_team=='Rajasthan Royals')&(df.bowler.isin(rr_spin)),'bowler'] = 'spinner'

In [15]:
df.loc[(df.bowling_team=='Royal Challengers Bangalore')&(df.bowler=='spinner')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
56,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,spinner,9.1,0,0,0,0,0,1,0,1,,1,"Joshi to Ponting, 1 run, some good turn, spins..."
57,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,spinner,9.2,0,0,0,0,0,1,0,1,,1,"Joshi to McCullum, 1 run, too straight and fla..."
58,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,spinner,9.3,0,0,0,0,0,1,0,1,,1,"Joshi to Ponting, 1 run, fired on leg again an..."
59,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,spinner,9.4,0,0,0,0,0,0,0,0,,1,"Joshi to McCullum, no run, bit of flight there..."
60,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,spinner,9.5,0,0,0,0,0,6,0,6,,1,"Joshi to McCullum, SIX runs, snuff is snuff, s..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130411,2019,53,Bangalore,1,Sunrisers Hyderabad,Royal Challengers Bangalore,spinner,15.2,0,0,0,0,0,1,0,1,,4,"Chahal to Williamson, 1 run, short of a length..."
130412,2019,53,Bangalore,1,Sunrisers Hyderabad,Royal Challengers Bangalore,spinner,15.3,0,0,0,0,0,1,0,1,,4,"Chahal to Pathan, 1 run, length at middle, nud..."
130413,2019,53,Bangalore,1,Sunrisers Hyderabad,Royal Challengers Bangalore,spinner,15.4,0,0,0,0,0,1,0,1,,4,"Chahal to Williamson, 1 run, length ball, slow..."
130414,2019,53,Bangalore,1,Sunrisers Hyderabad,Royal Challengers Bangalore,spinner,15.5,0,0,0,0,0,0,0,0,caught,5,"Chahal to Pathan, OUT, gets him! Once again, g..."


In [16]:
df.loc[(df.bowling_team=='Delhi Capitals')&(df.bowler=='spinner')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
540,2008,3,Delhi,1,Rajasthan Royals,Delhi Capitals,spinner,11.1,0,0,0,0,0,6,0,6,,5,"Vettori to Jadeja, SIX runs, and thats clobber..."
541,2008,3,Delhi,1,Rajasthan Royals,Delhi Capitals,spinner,11.2,0,0,0,0,0,4,0,4,,5,"Vettori to Jadeja, FOUR runs, smart batting, a..."
542,2008,3,Delhi,1,Rajasthan Royals,Delhi Capitals,spinner,11.3,0,0,0,0,0,1,0,1,,5,"Vettori to Jadeja, 1 run, looped up on off stu..."
543,2008,3,Delhi,1,Rajasthan Royals,Delhi Capitals,spinner,11.4,0,0,0,0,0,0,0,0,,5,"Vettori to Rawat, no run, well bowled, flighti..."
544,2008,3,Delhi,1,Rajasthan Royals,Delhi Capitals,spinner,11.5,0,0,0,0,0,1,0,1,,5,"Vettori to Rawat, 1 run, comes onto the front ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131731,2019,semi-final2,Visakhapatnam,2,Chennai Super Kings,Delhi Capitals,spinner,15.2,0,0,0,0,0,2,0,2,,2,"Patel to Raina, 2 runs, down the track and Axa..."
131732,2019,semi-final2,Visakhapatnam,2,Chennai Super Kings,Delhi Capitals,spinner,15.3,0,0,0,0,0,1,0,1,,2,"Patel to Raina, 1 run, flat, back of a length ..."
131733,2019,semi-final2,Visakhapatnam,2,Chennai Super Kings,Delhi Capitals,spinner,15.4,0,0,0,0,0,4,0,4,,2,"Patel to Rayudu, FOUR runs, shot. Down the tra..."
131734,2019,semi-final2,Visakhapatnam,2,Chennai Super Kings,Delhi Capitals,spinner,15.5,0,0,0,0,0,1,0,1,,2,"Patel to Rayudu, 1 run, flatter, on the pads, ..."


In [17]:
df.loc[(df.bowling_team=='Mumbai Indians')&(df.bowler=='spinner')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
858,2008,4,Mumbai,2,Royal Challengers Bangalore,Mumbai Indians,spinner,7.1,0,0,0,0,0,2,0,2,,1,"Jayasuriya to Dravid, 2 runs, flat on off stum..."
859,2008,4,Mumbai,2,Royal Challengers Bangalore,Mumbai Indians,spinner,7.2,0,0,0,0,0,1,0,1,,1,"Jayasuriya to Dravid, 1 run, slips down leg an..."
860,2008,4,Mumbai,2,Royal Challengers Bangalore,Mumbai Indians,spinner,7.3,0,0,0,0,0,1,0,1,,1,"Jayasuriya to Taylor, 1 run, flat and wide, sp..."
861,2008,4,Mumbai,2,Royal Challengers Bangalore,Mumbai Indians,spinner,7.4,0,0,0,0,0,1,0,1,,1,"Jayasuriya to Dravid, 1 run, flat on off stump..."
862,2008,4,Mumbai,2,Royal Challengers Bangalore,Mumbai Indians,spinner,7.5,0,0,0,0,0,1,0,1,,1,"Jayasuriya to Taylor, 1 run, flat on middle an..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131984,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,spinner,17.2,0,0,0,0,0,6,0,6,,4,"KH Pandya to Watson, SIX runs, hammered! Full,..."
131985,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,spinner,17.3,0,0,0,0,0,6,0,6,,4,"KH Pandya to Watson, SIX runs, over Krunal's h..."
131986,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,spinner,17.4,0,0,0,0,0,6,0,6,,4,"KH Pandya to Watson, SIX runs, into the second..."
131987,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,spinner,17.5,0,0,0,0,0,0,0,0,,4,"KH Pandya to Watson, no run, full, fast, and w..."


In [18]:
df.loc[(df.bowling_team=='Chennai Super Kings')&(df.bowler=='spinner')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
387,2008,2,Mohali,2,Kings XI Punjab,Chennai Super Kings,spinner,6.1,0,0,0,0,0,1,0,1,,1,"Muralitharan to Sangakkara, 1 run, full and on..."
388,2008,2,Mohali,2,Kings XI Punjab,Chennai Super Kings,spinner,6.2,0,0,0,0,0,1,0,1,,1,"Muralitharan to Hopes, 1 run, full and swept a..."
389,2008,2,Mohali,2,Kings XI Punjab,Chennai Super Kings,spinner,6.3,0,0,0,0,0,1,0,1,,1,"Muralitharan to Sangakkara, 1 run, down the tr..."
390,2008,2,Mohali,2,Kings XI Punjab,Chennai Super Kings,spinner,6.4,0,0,0,0,0,1,0,1,,1,"Muralitharan to Hopes, 1 run, full toss but Ho..."
391,2008,2,Mohali,2,Kings XI Punjab,Chennai Super Kings,spinner,6.5,0,0,1,0,0,0,1,1,,1,"Muralitharan to Sangakkara, 1 leg bye, down th..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131854,2019,final,Hyderabad,1,Mumbai Indians,Chennai Super Kings,spinner,16.2,0,0,0,0,0,1,0,1,,5,"Imran Tahir to Pollard, 1 run, slider, length ..."
131855,2019,final,Hyderabad,1,Mumbai Indians,Chennai Super Kings,spinner,16.3,0,0,0,0,0,1,0,1,,5,"Imran Tahir to HH Pandya, 1 run, yorker on off..."
131856,2019,final,Hyderabad,1,Mumbai Indians,Chennai Super Kings,spinner,16.4,0,0,0,0,0,0,0,0,,5,"Imran Tahir to Pollard, no run, full on off, a..."
131857,2019,final,Hyderabad,1,Mumbai Indians,Chennai Super Kings,spinner,16.5,0,0,0,0,0,6,0,6,,5,"Imran Tahir to Pollard, SIX runs, going, going..."


In [19]:
df.loc[(df.bowling_team=='Rajasthan Royals')&(df.bowler=='spinner')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
646,2008,3,Delhi,2,Delhi Capitals,Rajasthan Royals,spinner,8.1,0,0,0,0,0,0,0,0,,1,"Warne to Gambhir, no run, legbreak on the stum..."
647,2008,3,Delhi,2,Delhi Capitals,Rajasthan Royals,spinner,8.2,0,0,0,0,0,1,0,1,,1,"Warne to Gambhir, 1 run, a tad fuller and Gamb..."
648,2008,3,Delhi,2,Delhi Capitals,Rajasthan Royals,spinner,8.3,0,0,0,0,0,4,0,4,,1,"Warne to Dhawan, FOUR runs, shot! Flighted up ..."
649,2008,3,Delhi,2,Delhi Capitals,Rajasthan Royals,spinner,8.4,0,0,0,0,0,0,0,0,,1,"Warne to Dhawan, no run, flatter trajectory an..."
650,2008,3,Delhi,2,Delhi Capitals,Rajasthan Royals,spinner,8.5,0,0,0,0,0,1,0,1,,1,"Warne to Dhawan, 1 run, this time he connects ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130313,2019,52,Delhi,2,Delhi Capitals,Rajasthan Royals,spinner,15.3,0,0,0,0,0,1,0,1,,5,"Gopal to Pant, 1 run, googly, short outside of..."
130314,2019,52,Delhi,2,Delhi Capitals,Rajasthan Royals,spinner,15.4,0,0,0,0,0,1,0,1,,5,"Gopal to Patel, 1 run, shorter length, he's ba..."
130315,2019,52,Delhi,2,Delhi Capitals,Rajasthan Royals,spinner,15.5,0,0,0,0,0,6,0,6,,5,"Gopal to Pant, SIX runs, thumped over long on!..."
130316,2019,52,Delhi,2,Delhi Capitals,Rajasthan Royals,spinner,15.6,0,0,0,0,0,1,0,1,,5,"Gopal to Pant, 1 run, shorter length, he's bac..."


In [20]:
df.loc[(df.bowling_team=='Deccan Chargers')&(df.bowler=='spinner')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
1134,2008,5,Kolkata,2,Kolkata Knight Riders,Deccan Chargers,spinner,12.1,1,0,0,0,0,0,1,1,,3,"Ojha to Ganguly, 1 wide, starts off with a wid..."
1135,2008,5,Kolkata,2,Kolkata Knight Riders,Deccan Chargers,spinner,12.1,0,0,0,0,0,0,0,0,,3,"Ojha to Ganguly, no run, flighted delivery, tu..."
1136,2008,5,Kolkata,2,Kolkata Knight Riders,Deccan Chargers,spinner,12.2,0,0,0,0,0,0,0,0,,3,"Ojha to Ganguly, no run, fuller and driven har..."
1137,2008,5,Kolkata,2,Kolkata Knight Riders,Deccan Chargers,spinner,12.3,0,0,0,0,0,1,0,1,,3,"Ojha to Ganguly, 1 run, charges out but is don..."
1138,2008,5,Kolkata,2,Kolkata Knight Riders,Deccan Chargers,spinner,12.4,1,0,0,0,0,0,1,1,,3,"Ojha to Hussey, 1 wide, lands outside leg, tur..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51189,2012,53,Hyderabad,2,Royal Challengers Bangalore,Deccan Chargers,spinner,17.2,0,0,0,0,0,1,0,1,,6,"Mishra to Vinay Kumar, 1 run, driven off the f..."
51190,2012,53,Hyderabad,2,Royal Challengers Bangalore,Deccan Chargers,spinner,17.3,0,0,0,0,0,2,0,2,,6,"Mishra to Tiwary, 2 runs, a flat and fast ball..."
51191,2012,53,Hyderabad,2,Royal Challengers Bangalore,Deccan Chargers,spinner,17.4,0,0,0,0,0,2,0,2,,6,"Mishra to Tiwary, 2 runs, Tiwary takes a step ..."
51192,2012,53,Hyderabad,2,Royal Challengers Bangalore,Deccan Chargers,spinner,17.5,0,0,0,0,0,1,0,1,,6,"Mishra to Tiwary, 1 run, swung off the pads be..."


In [21]:
df.loc[(df.bowling_team=='Kings XI Punjab')&(df.bowler=='spinner')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
287,2008,2,Mohali,1,Chennai Super Kings,Kings XI Punjab,spinner,10.1,0,0,0,0,0,6,0,6,,3,"Chawla to Hussey, SIX runs, over the wicket an..."
288,2008,2,Mohali,1,Chennai Super Kings,Kings XI Punjab,spinner,10.2,0,0,0,0,0,1,0,1,,3,"Chawla to Hussey, 1 run, quicker one this time..."
289,2008,2,Mohali,1,Chennai Super Kings,Kings XI Punjab,spinner,10.3,0,0,0,0,0,2,0,2,,3,"Chawla to Raina, 2 runs, quicker again, outsid..."
290,2008,2,Mohali,1,Chennai Super Kings,Kings XI Punjab,spinner,10.4,0,0,0,0,0,0,0,0,,3,"Chawla to Raina, no run, better ball that, str..."
291,2008,2,Mohali,1,Chennai Super Kings,Kings XI Punjab,spinner,10.5,0,0,0,0,0,6,0,6,,3,"Chawla to Raina, SIX runs, huge! short and dri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130668,2019,54,Mohali,1,Chennai Super Kings,Kings XI Punjab,spinner,17.2,0,0,0,0,0,1,0,1,,2,"R Ashwin to Dhoni, 1 run, short carrom ball, h..."
130669,2019,54,Mohali,1,Chennai Super Kings,Kings XI Punjab,spinner,17.3,0,0,0,0,0,1,0,1,,2,"R Ashwin to du Plessis, 1 run, tossed up, he s..."
130670,2019,54,Mohali,1,Chennai Super Kings,Kings XI Punjab,spinner,17.4,0,0,0,0,0,0,0,0,,2,"R Ashwin to Dhoni, no run, shorter length, wid..."
130671,2019,54,Mohali,1,Chennai Super Kings,Kings XI Punjab,spinner,17.5,0,0,0,0,0,0,0,0,,2,"R Ashwin to Dhoni, no run, leg break, he just ..."


In [22]:
df.loc[(df.bowling_team=='Kolkata Knight Riders')&(df.bowler=='spinner')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
977,2008,5,Kolkata,1,Deccan Chargers,Kolkata Knight Riders,spinner,6.1,0,0,0,0,0,1,0,1,,2,"Kartik to Symonds, 1 run, on the middle and le..."
978,2008,5,Kolkata,1,Deccan Chargers,Kolkata Knight Riders,spinner,6.2,0,0,0,0,0,0,0,0,,2,"Kartik to Gilchrist, no run, turns in towards ..."
979,2008,5,Kolkata,1,Deccan Chargers,Kolkata Knight Riders,spinner,6.3,0,0,0,0,0,1,0,1,,2,"Kartik to Gilchrist, 1 run, spins towards the ..."
980,2008,5,Kolkata,1,Deccan Chargers,Kolkata Knight Riders,spinner,6.4,0,0,0,0,0,4,0,4,,2,"Kartik to Symonds, FOUR runs, flighted one, fu..."
981,2008,5,Kolkata,1,Deccan Chargers,Kolkata Knight Riders,spinner,6.5,0,0,0,0,0,0,0,0,,2,"Kartik to Symonds, no run, Now then, that's a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131008,2019,55,Mumbai,2,Mumbai Indians,Kolkata Knight Riders,spinner,13.2,0,0,0,0,0,6,0,6,,1,"Narine to Yadav, SIX runs, deposited into the ..."
131009,2019,55,Mumbai,2,Mumbai Indians,Kolkata Knight Riders,spinner,13.3,0,0,0,0,0,1,0,1,,1,"Narine to Yadav, 1 run, flicked with the turn ..."
131010,2019,55,Mumbai,2,Mumbai Indians,Kolkata Knight Riders,spinner,13.4,0,0,0,0,0,0,0,0,,1,"Narine to Sharma, no run, Narine zones in on a..."
131011,2019,55,Mumbai,2,Mumbai Indians,Kolkata Knight Riders,spinner,13.5,0,0,0,0,0,0,0,0,,1,"Narine to Sharma, no run, Rohit's down the tra..."


In [23]:
df.loc[(df.bowling_team=='Sunrisers Hyderabad')&(df.bowler=='spinner')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
53441,2013,5,Hyderabad,1,Royal Challengers Bangalore,Sunrisers Hyderabad,spinner,4.1,0,0,0,0,0,5,0,5,,2,"Ankit Sharma to Nair, 5 runs, flights it up to..."
53442,2013,5,Hyderabad,1,Royal Challengers Bangalore,Sunrisers Hyderabad,spinner,4.2,0,0,0,0,0,0,0,0,,2,"Ankit Sharma to Kohli, no run, tossed up and s..."
53443,2013,5,Hyderabad,1,Royal Challengers Bangalore,Sunrisers Hyderabad,spinner,4.3,0,0,0,0,0,0,0,0,,2,"Ankit Sharma to Kohli, no run, beautiful deliv..."
53444,2013,5,Hyderabad,1,Royal Challengers Bangalore,Sunrisers Hyderabad,spinner,4.4,0,0,0,0,0,4,0,4,,2,"Ankit Sharma to Kohli, FOUR runs, he has had e..."
53445,2013,5,Hyderabad,1,Royal Challengers Bangalore,Sunrisers Hyderabad,spinner,4.5,0,0,0,0,0,1,0,1,,2,"Ankit Sharma to Kohli, 1 run, shortens the len..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131485,2019,eliminator,Visakhapatnam,2,Delhi Capitals,Sunrisers Hyderabad,spinner,15.2,0,0,0,0,0,0,0,0,,4,"Mohammad Nabi to Rutherford, no run, length, a..."
131486,2019,eliminator,Visakhapatnam,2,Delhi Capitals,Sunrisers Hyderabad,spinner,15.3,0,0,0,0,0,0,0,0,,4,"Mohammad Nabi to Rutherford, no run, turns fro..."
131487,2019,eliminator,Visakhapatnam,2,Delhi Capitals,Sunrisers Hyderabad,spinner,15.4,0,0,0,0,0,1,0,1,,4,"Mohammad Nabi to Rutherford, 1 run, length, an..."
131488,2019,eliminator,Visakhapatnam,2,Delhi Capitals,Sunrisers Hyderabad,spinner,15.5,0,0,0,0,0,6,0,6,,4,"Mohammad Nabi to Pant, SIX runs, length, and h..."


In [24]:
l = ['spinner']
df.loc[~df.bowler.isin(l),'bowler'] = 'pacer'

In [25]:
df.loc[df.bowler=='pacer']

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
0,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.1,0,0,1,0,0,0,1,1,,0,"Kumar to Ganguly, 1 leg bye, and thats runs im..."
1,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.2,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, this one nips back ..."
2,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.3,1,0,0,0,0,0,1,1,,0,"Kumar to McCullum, 1 wide, thats wide, as Prav..."
3,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.3,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, too straight, McCul..."
4,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.4,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, steps down the trac..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131996,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.2,0,0,0,0,0,1,0,1,,5,"Malinga to Jadeja, 1 run, low full toss on mid..."
131997,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.3,0,0,0,0,0,2,0,2,,5,"Malinga to Watson, 2 runs, yorker, just outsid..."
131998,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.4,0,0,0,0,0,1,0,1,run out,6,"Malinga to Watson, 1 run, OUT, yorker, just ou..."
131999,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.5,0,0,0,0,0,2,0,2,,6,"Malinga to Thakur, 2 runs, full toss on leg, a..."


In [26]:
df.loc[df.bowler=='spinner']

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
56,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,spinner,9.1,0,0,0,0,0,1,0,1,,1,"Joshi to Ponting, 1 run, some good turn, spins..."
57,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,spinner,9.2,0,0,0,0,0,1,0,1,,1,"Joshi to McCullum, 1 run, too straight and fla..."
58,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,spinner,9.3,0,0,0,0,0,1,0,1,,1,"Joshi to Ponting, 1 run, fired on leg again an..."
59,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,spinner,9.4,0,0,0,0,0,0,0,0,,1,"Joshi to McCullum, no run, bit of flight there..."
60,2008,1,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,spinner,9.5,0,0,0,0,0,6,0,6,,1,"Joshi to McCullum, SIX runs, snuff is snuff, s..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131984,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,spinner,17.2,0,0,0,0,0,6,0,6,,4,"KH Pandya to Watson, SIX runs, hammered! Full,..."
131985,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,spinner,17.3,0,0,0,0,0,6,0,6,,4,"KH Pandya to Watson, SIX runs, over Krunal's h..."
131986,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,spinner,17.4,0,0,0,0,0,6,0,6,,4,"KH Pandya to Watson, SIX runs, into the second..."
131987,2019,final,Hyderabad,2,Chennai Super Kings,Mumbai Indians,spinner,17.5,0,0,0,0,0,0,0,0,,4,"KH Pandya to Watson, no run, full, fast, and w..."


In [27]:
playoff = ['3rd Place Playoff', 'eliminator', 'final', 'semi-final1', 'semi-final2', 'semi-final']
df.loc[df.match_id.isin(playoff), 'match_id'] = 'playoffs'

In [28]:
df.loc[df.match_id=='playoffs']

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
12801,2008,playoffs,Mumbai,1,Rajasthan Royals,Delhi Capitals,pacer,0.1,0,0,0,0,0,0,0,0,,0,"McGrath to Smith, no run, a peach from McGrath..."
12802,2008,playoffs,Mumbai,1,Rajasthan Royals,Delhi Capitals,pacer,0.2,1,0,0,0,0,0,1,1,,0,"McGrath to Smith, 1 wide, and this one is wide..."
12803,2008,playoffs,Mumbai,1,Rajasthan Royals,Delhi Capitals,pacer,0.2,0,0,0,0,0,0,0,0,,0,"McGrath to Smith, no run, tight line and lengt..."
12804,2008,playoffs,Mumbai,1,Rajasthan Royals,Delhi Capitals,pacer,0.3,0,0,0,0,0,0,0,0,,0,"McGrath to Smith, no run, good length once aga..."
12805,2008,playoffs,Mumbai,1,Rajasthan Royals,Delhi Capitals,pacer,0.4,0,0,0,0,0,0,0,0,,0,"McGrath to Smith, no run, Smith tries to flick..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131996,2019,playoffs,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.2,0,0,0,0,0,1,0,1,,5,"Malinga to Jadeja, 1 run, low full toss on mid..."
131997,2019,playoffs,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.3,0,0,0,0,0,2,0,2,,5,"Malinga to Watson, 2 runs, yorker, just outsid..."
131998,2019,playoffs,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.4,0,0,0,0,0,1,0,1,run out,6,"Malinga to Watson, 1 run, OUT, yorker, just ou..."
131999,2019,playoffs,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.5,0,0,0,0,0,2,0,2,,6,"Malinga to Thakur, 2 runs, full toss on leg, a..."


In [29]:
p = ['playoffs']
df.loc[~df.match_id.isin(p),'match_id'] = 'round-robin'

In [30]:
df.loc[df.match_id=='round-robin']

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
0,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.1,0,0,1,0,0,0,1,1,,0,"Kumar to Ganguly, 1 leg bye, and thats runs im..."
1,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.2,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, this one nips back ..."
2,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.3,1,0,0,0,0,0,1,1,,0,"Kumar to McCullum, 1 wide, thats wide, as Prav..."
3,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.3,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, too straight, McCul..."
4,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.4,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, steps down the trac..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131021,2019,round-robin,Mumbai,2,Mumbai Indians,Kolkata Knight Riders,pacer,15.3,0,0,0,0,0,0,0,0,,1,"Prasidh Krishna to Sharma, no run,Ê"
131022,2019,round-robin,Mumbai,2,Mumbai Indians,Kolkata Knight Riders,pacer,15.4,0,0,0,0,0,4,0,4,,1,"Prasidh Krishna to Sharma, FOUR runs, chipped ..."
131023,2019,round-robin,Mumbai,2,Mumbai Indians,Kolkata Knight Riders,pacer,15.5,0,0,0,0,0,1,0,1,,1,"Prasidh Krishna to Sharma, 1 run, very full, j..."
131024,2019,round-robin,Mumbai,2,Mumbai Indians,Kolkata Knight Riders,pacer,15.6,0,0,0,0,0,1,0,1,,1,"Prasidh Krishna to Yadav, 1 run, Prasidh digs ..."


In [31]:
df.rename(columns={"match_id": "match_type", "bowler": "bowler_type"})

,season,match_type,venue,inning,batting_team,bowling_team,bowler_type,ball,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
0,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.1,0,0,1,0,0,0,1,1,,0,"Kumar to Ganguly, 1 leg bye, and thats runs im..."
1,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.2,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, this one nips back ..."
2,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.3,1,0,0,0,0,0,1,1,,0,"Kumar to McCullum, 1 wide, thats wide, as Prav..."
3,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.3,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, too straight, McCul..."
4,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.4,0,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, steps down the trac..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131996,2019,playoffs,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.2,0,0,0,0,0,1,0,1,,5,"Malinga to Jadeja, 1 run, low full toss on mid..."
131997,2019,playoffs,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.3,0,0,0,0,0,2,0,2,,5,"Malinga to Watson, 2 runs, yorker, just outsid..."
131998,2019,playoffs,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.4,0,0,0,0,0,1,0,1,run out,6,"Malinga to Watson, 1 run, OUT, yorker, just ou..."
131999,2019,playoffs,Hyderabad,2,Chennai Super Kings,Mumbai Indians,pacer,19.5,0,0,0,0,0,2,0,2,,6,"Malinga to Thakur, 2 runs, full toss on leg, a..."


In [32]:
df.drop(labels='penalty_runs', axis=1, inplace=True)
df.drop(labels=1614, axis=0, inplace=True)

In [33]:
df.head()

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary
0,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.1,0,0,1,0,0,1,1,,0,"Kumar to Ganguly, 1 leg bye, and thats runs im..."
1,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.2,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, this one nips back ..."
2,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.3,1,0,0,0,0,1,1,,0,"Kumar to McCullum, 1 wide, thats wide, as Prav..."
3,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.3,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, too straight, McCul..."
4,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.4,0,0,0,0,0,0,0,,0,"Kumar to McCullum, no run, steps down the trac..."


In [34]:
import re
def lower_comm(x):
    return x.lower()
df['commentary'] = df['commentary'].apply(lower_comm)

In [35]:
full_toss_length = ['full toss', 'beamer', 'full-toss', 'fulltoss']
yorker_length = ['yorker', 'yorker length', 'yorker-ish', 'blockhole', 'block-hole', 'block hole', 'toe-crusher', 'toe crusher']
full_length = ['full', 'full and straight', 'full length', 'fullish', 'full-ish', 'fuller', 'very full', 'full ball', 'full delivery', 'forward', 'front foot', 'flicked off the pads', 'glanced off the pads', 'flicks off the pad', 'off the pads', 'off the pad',
               'clips', 'clipped', 'pads', 'on the pad', 'into the pads', 'cover drive', 'front', 'beaten', 'beaten on the drive','pitches it up', 'full on', 'slog', 'slogs', 'flat and quick',
               'onto the pads', 'aimed at leg stump', 'aimed at leg-stump', 'pitched at leg stump', 'floated up', 'floats', 'floating', 'flights', 'flight', 'inside out', 'in the slot', 'slot delivery', 'long on', 'long-on',
               'flighted', 'flighting', 'tossed up', 'overpitched', 'overpitching', 'overpitches', 'pitched up', 'half-volley', 'half volley', 'loopy', 'looping', 'reverse sweep', 'down the ground',
               'reverse-sweep', 'sweep', 'sweeping', 'paddled', 'paddle', 'paddles', 'slog swept', 'slog-sweep', 'off-drive', 'straight-drive', 'worked', 'works', 'meeting the pitch', 'lunges forward', 'cover-drive', 'cover drive',
               'off drive', 'slog-sweeps', 'straight drive', 'on-drive', 'on drive', 'helicopter shot', 'helicopter', 'over cover', 'over extra cover', 'pitch of the ball', 'to the pitch', 'slower full', 'slower, full']
good_length = ['length ball', 'good length', 'length delivery', 'at a length', ', length,', 'length', 'peach', 'corridor of uncertainty', 'punched', 'googly', 'cramped', 'cramps', 'cramping', 'thigh pad', 'cramp',
               'goodish', 'good-ish', 'nudges', 'driven on the up', 'slower length ball', 'slower, good length', 'on the up', 'length outside', 'length on', 'tight length']
short_length = ['short', 'short ball', 'short delivery', 'bouncer', 'pulled', 'hooked', 'pulling', 'hooking', 'pull', 'hook', 
                'back of a length', 'back of length', 'back of good length', 'back foot', 'back-foot', 'heaved', 'hoicked',
                'short of good length', 'short length', 'shortish', 'short-ish', 'cuts', 'cut', 'cutting it', 'cutting the', 'dragged back', 'drags his length back',
                'back-of-a-length', 'holding back the length']

In [36]:
conditions  = [df.commentary.str.contains('|'.join(full_toss_length)), df.commentary.str.contains('|'.join(yorker_length)), df.commentary.str.contains('|'.join(full_length)), df.commentary.str.contains('|'.join(good_length)), df.commentary.str.contains('|'.join(short_length))]
choices = ["full toss", "yorker", "full", "good", "short"]

df["ball_length"] = np.select(conditions, choices, default=np.nan)
df.head(50)

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary,ball_length
0,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.1,0,0,1,0,0,1,1,,0,"kumar to ganguly, 1 leg bye, and thats runs im...",full
1,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.2,0,0,0,0,0,0,0,,0,"kumar to mccullum, no run, this one nips back ...",good
2,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.3,1,0,0,0,0,1,1,,0,"kumar to mccullum, 1 wide, thats wide, as prav...",nan
3,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.3,0,0,0,0,0,0,0,,0,"kumar to mccullum, no run, too straight, mccul...",nan
4,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.4,0,0,0,0,0,0,0,,0,"kumar to mccullum, no run, steps down the trac...",good
5,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.5,0,0,0,0,0,0,0,,0,"kumar to mccullum, no run, good bowling, nice ...",good
6,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.6,0,0,1,0,0,1,1,,0,"kumar to mccullum, 1 leg bye, shorter one slid...",full
7,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,1.1,0,0,0,0,0,0,0,,0,"khan to mccullum, no run, shuffles around in h...",short
8,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,1.2,0,0,0,0,4,0,4,,0,"khan to mccullum, four runs, bang, thats four!...",nan
9,2008,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,1.3,0,0,0,0,4,0,4,,0,"khan to mccullum, four runs, shot, man! zaheer...",short


## STOP AND GO

In [ ]:
df['batsman_runs'] = df['batsman_runs'].astype('int64')
df['total_runs'] = df['total_runs'].astype('int64')
df['extra_runs'] = df['extra_runs'].astype('int64')
df['noball_runs'] = df['noball_runs'].astype('int64')
df['legbye_runs'] = df['legbye_runs'].astype('int64')
df['bye_runs'] = df['bye_runs'].astype('int64')
df['wide_runs'] = df['wide_runs'].astype('int64')

def event_pred(row):
    if row['batsman_runs']== 0 and row['extra_runs']== 0 and row['total_runs'] == 0 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return '0 Runs'
    if row['batsman_runs']==1 and row['extra_runs']==0 and row['total_runs'] == 1 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return '1 Run'
    if row['batsman_runs']==2 and row['extra_runs']==0 and row['total_runs'] == 2 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return '2 Runs'
    if row['batsman_runs']==3 and row['extra_runs']==0 and row['total_runs'] == 3 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return '3 Runs'
    if row['batsman_runs']==4 and row['extra_runs']==0 and row['total_runs'] == 4 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return '4 Runs!'
    if row['batsman_runs']==5 and row['extra_runs']==0 and row['total_runs'] == 5 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return '5 Runs'
    if row['batsman_runs']==6 and row['extra_runs']==0 and row['total_runs'] == 6 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return '6 Runs!'
    if row['wide_runs']>0 and row['extra_runs']>0 and row['total_runs']>0 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Extras (wide)'
    if row['bye_runs']>0 and row['extra_runs']>0 and row['total_runs']>0 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Extras (bye)'
    if row['legbye_runs']>0 and row['extra_runs']>0 and row['total_runs']>0 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Extras (legbye)'
    if row['noball_runs']>0 and row['extra_runs']>0 and row['batsman_runs']==0 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Extras (noball) + 0 Batsman Runs'
    if row['noball_runs']>0 and row['extra_runs']>0 and row['batsman_runs']==1 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Extras (noball) + 1 Batsman Run'
    if row['noball_runs']>0 and row['extra_runs']>0 and row['batsman_runs']==2 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Extras (noball) + 2 Batsman Runs'
    if row['noball_runs']>0 and row['extra_runs']>0 and row['batsman_runs']==3 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Extras (noball) + 3 Batsman Runs'
    if row['noball_runs']>0 and row['extra_runs']>0 and row['batsman_runs']==4 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Extras (noball) + 4 Batsman Runs!'
    if row['noball_runs']>0 and row['extra_runs']>0 and row['batsman_runs']==6 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Extras (noball) + 6 Batsman Runs!'
    if row['dismissal_kind']=='bowled':
        return 'WICKET (Bowled)!'
    if row['dismissal_kind']=='caught':
        return 'WICKET (Caught)!'
    if row['dismissal_kind']=='caught and bowled':
        return 'WICKET (Caught & Bowled)!'
    if row['dismissal_kind']=='hit wicket':
        return 'WICKET (Hit Wicket)!'
    if row['dismissal_kind']=='lbw':
        return 'WICKET (LBW - Leg Before Wicket)!'
    if row['dismissal_kind']=='obstructing the field':
        return 'WICKET (Obstructing the Field)!'
    if row['dismissal_kind']=='retired hurt':
        return 'Retired Hurt'
    if row['dismissal_kind']=='run out':
        return 'WICKET (Run Out)!'
    if row['dismissal_kind']=='stumped':
        return 'WICKET (Stumped)!'

In [37]:
df['batsman_runs'] = df['batsman_runs'].astype('int64')
df['total_runs'] = df['total_runs'].astype('int64')
df['extra_runs'] = df['extra_runs'].astype('int64')
df['noball_runs'] = df['noball_runs'].astype('int64')
df['legbye_runs'] = df['legbye_runs'].astype('int64')
df['bye_runs'] = df['bye_runs'].astype('int64')
df['wide_runs'] = df['wide_runs'].astype('int64')

def event_pred(row):
    if row['batsman_runs']==0 and row['extra_runs']== 0 and row['total_runs'] == 0 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Dot'
    if row['batsman_runs']>0 and row['extra_runs']== 0 and row['total_runs'] > 0 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Runs'
    if row['wide_runs']>0 or row['bye_runs']>0 or row['legbye_runs']>0 or row['noball_runs']>0 or row['batsman_runs']>=0 and row['extra_runs']>0 and row['total_runs']>0 and row['dismissal_kind']!='bowled' and row['dismissal_kind']!='caught' and row['dismissal_kind']!='caught and bowled' and row['dismissal_kind']!='hit wicket' and row['dismissal_kind']!='lbw' and row['dismissal_kind']!='obstructing the field' and row['dismissal_kind']!='retired hurt' and row['dismissal_kind']!='run out' and row['dismissal_kind']!='stumped':
        return 'Extras'
    if row['dismissal_kind']=='bowled' or row['dismissal_kind']=='caught' or row['dismissal_kind']=='caught and bowled' or row['dismissal_kind']=='hit wicket' or row['dismissal_kind']=='lbw' or row['dismissal_kind']=='obstructing the field' or row['dismissal_kind']=='retired hurt' or row['dismissal_kind']=='run out' or row['dismissal_kind']=='stumped':
        return 'WICKET!'

In [38]:
df['event'] = df.apply(lambda row: event_pred(row), axis=1)

In [39]:
df.loc[df.ball_length=='nan'].shape[0]

25123

In [ ]:
#df.loc[(df.ball_length=='nan') & (df.bowler=='spinner'), 'ball_length'] = 'good'

In [ ]:
#df.loc[(df.ball_length=='nan') & (df.bowler=='spinner')]

In [ ]:
#df.loc[(df.ball_length=='nan') & (df.event=='Extras (wide)'), 'ball_length'] = 'good'

In [ ]:
#df.loc[(df.ball_length=='nan') & (df.event=='Extras (legbye)'), 'ball_length'] = 'full'

In [ ]:
#df.loc[(df.ball_length=='nan') & (df.event=='0 Runs') | (df.event=='1 Run') | (df.event=='2 Runs') | (df.event=='3 Runs'), 'ball_length'] = 'good'

In [ ]:
#df.loc[(df.ball_length=='nan') & (df.event=='4 Runs!') |  (df.event=='6 Runs!'), 'ball_length'] = 'full'

In [ ]:
#df.loc[(df.ball_length=='nan') & (df.event=='5 Runs') | (df.event=='Extras (bye)') | (df.event=='Extras (noball) + 0 Batsman Runs'), 'ball_length'] = 'good'

In [ ]:
#df.loc[(df.ball_length=='nan') & (df.event=='Extras (noball) + 6 Batsman Runs!') | (df.event=='Extras (noball) + 4 Batsman Runs!') | (df.event=='Extras (noball) + 1 Batsman Run') | (df.event=='Extras (noball) + 2 Batsman Runs'), 'ball_length'] = 'full'

In [ ]:
#df.loc[(df.ball_length=='nan') & (df.event=='WICKET (Caught)!') | (df.event=='WICKET (Run Out)!') | (df.event=='WICKET (Hit Wicket)!'), 'ball_length'] = 'good'
#df.loc[(df.ball_length=='nan') & (df.event=='WICKET (Bowled)!') | (df.event=='WICKET (LBW - Leg Before Wicket)!') | (df.event=='WICKET (Caught & Bowled)!') | (df.event=='WICKET (Stumped)!'), 'ball_length'] = 'full'

In [40]:
df.drop(df.loc[df.ball_length=='nan'].index, inplace=True)

In [41]:
df.loc[(df.ball_length=='nan')]

,season,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wide_runs,bye_runs,legbye_runs,noball_runs,batsman_runs,extra_runs,total_runs,dismissal_kind,wickets,commentary,ball_length,event


In [42]:
df.loc[df.event=='Dot'].shape[0]

31431

In [43]:
df.loc[df.event=='Runs'].shape[0]

65315

In [44]:
df.loc[df.event=='Extras'].shape[0]

4637

In [45]:
df.loc[df.event=='WICKET!'].shape[0]

5494

In [ ]:
df.dtypes

In [46]:
df['match_id'] = df['match_id'].astype('category')
df['venue'] = df['venue'].astype('category')
df['inning'] = df['inning'].astype('category')
df['batting_team'] = df['batting_team'].astype('category')
df['bowling_team'] = df['bowling_team'].astype('category')
df['bowler'] = df['bowler'].astype('category')
df['ball_length'] = df['ball_length'].astype('category')

df['event'] = df['event'].astype('object')

df['ball'] = df['ball'].astype('float64')
     
df['wickets'] = df['wickets'].astype('int64')  

df.drop(labels='commentary', axis=1, inplace=True)
df.drop(labels='wide_runs', axis=1, inplace=True)
df.drop(labels='bye_runs', axis=1, inplace=True)
df.drop(labels='legbye_runs', axis=1, inplace=True)
df.drop(labels='noball_runs', axis=1, inplace=True)
df.drop(labels='batsman_runs', axis=1, inplace=True)
df.drop(labels='extra_runs', axis=1, inplace=True)
df.drop(labels='total_runs', axis=1, inplace=True)
df.drop(labels='dismissal_kind', axis=1, inplace=True)
df.drop(labels='season', axis=1, inplace=True)

df.head()

,match_id,venue,inning,batting_team,bowling_team,bowler,ball,wickets,ball_length,event
0,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.1,0,full,Extras
1,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.2,0,good,Dot
4,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.4,0,good,Dot
5,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.5,0,good,Dot
6,round-robin,Bangalore,1,Kolkata Knight Riders,Royal Challengers Bangalore,pacer,0.6,0,full,Extras


In [ ]:
df.dtypes

In [ ]:
df.describe()

In [47]:
from sklearn.preprocessing import minmax_scale
df[['ball','wickets']] = minmax_scale(df[['ball','wickets']])

encoded_df = pd.get_dummies(data=df, columns=['match_id', 'venue', 'inning', 'batting_team', 'bowling_team', 'bowler', 'ball_length'])

In [52]:
encoded_df.columns

Index(['ball', 'wickets', 'match_id_playoffs', 'match_id_round-robin',
       'venue_Ahmedabad', 'venue_Bangalore', 'venue_Chennai', 'venue_Cuttack',
       'venue_Delhi', 'venue_Dharamsala', 'venue_Hyderabad', 'venue_Indore',
       'venue_Kolkata', 'venue_Mohali', 'venue_Mumbai', 'venue_Nagpur',
       'venue_Pune', 'venue_Raipur', 'venue_Rajasthan', 'venue_Ranchi',
       'venue_Visakhapatnam', 'inning_1', 'inning_2',
       'batting_team_Chennai Super Kings', 'batting_team_Deccan Chargers',
       'batting_team_Delhi Capitals', 'batting_team_Kings XI Punjab',
       'batting_team_Kolkata Knight Riders', 'batting_team_Mumbai Indians',
       'batting_team_Rajasthan Royals',
       'batting_team_Royal Challengers Bangalore',
       'batting_team_Sunrisers Hyderabad', 'bowling_team_Chennai Super Kings',
       'bowling_team_Deccan Chargers', 'bowling_team_Delhi Capitals',
       'bowling_team_Kings XI Punjab', 'bowling_team_Kolkata Knight Riders',
       'bowling_team_Mumbai Indians',

In [ ]:
encoded_df.shape

In [48]:
encoded_df = encoded_df[['ball', 'wickets', 'match_id_playoffs',
       'match_id_round-robin', 'venue_Ahmedabad', 'venue_Bangalore', 
       'venue_Chennai', 'venue_Cuttack', 'venue_Delhi',
       'venue_Dharamsala', 'venue_Hyderabad', 'venue_Indore', 'venue_Kolkata',
       'venue_Mohali', 'venue_Mumbai', 'venue_Nagpur', 'venue_Pune',
       'venue_Raipur', 'venue_Rajasthan', 'venue_Ranchi',
       'venue_Visakhapatnam', 'inning_1', 'inning_2',
       'batting_team_Chennai Super Kings', 'batting_team_Deccan Chargers',
       'batting_team_Delhi Capitals', 'batting_team_Kings XI Punjab',
       'batting_team_Kolkata Knight Riders', 'batting_team_Mumbai Indians',
       'batting_team_Rajasthan Royals',
       'batting_team_Royal Challengers Bangalore',
       'batting_team_Sunrisers Hyderabad', 'bowling_team_Chennai Super Kings',
       'bowling_team_Deccan Chargers', 'bowling_team_Delhi Capitals',
       'bowling_team_Kings XI Punjab', 'bowling_team_Kolkata Knight Riders',
       'bowling_team_Mumbai Indians', 'bowling_team_Rajasthan Royals',
       'bowling_team_Royal Challengers Bangalore',
       'bowling_team_Sunrisers Hyderabad', 'bowler_pacer', 'bowler_spinner',
       'ball_length_full', 'ball_length_full toss', 'ball_length_good',
       'ball_length_short', 'ball_length_yorker', 'event']]

In [53]:
encoded_df['event'].unique()

array([1, 0, 2, 3])

In [49]:
X = encoded_df.iloc[:,0:48].values
X.shape

(106877, 48)

In [50]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoded_df['event'] = encoder.fit_transform(encoded_df['event'])
Y = pd.get_dummies(encoded_df['event']).values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.213530303,random_state=0)

In [54]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers

In [55]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(48,), activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                1568      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 68        
Total params: 2,164
Trainable params: 2,164
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [60]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
class_weight = {0: 4.,
                1: 180.,
                2: 1.,
                3: 180.}
model.fit(X_train,y_train,batch_size=32,epochs=20)

Epoch 1/20
2627/2627 [==============================] - 3s 1ms/step - loss: 0.8943 - accuracy: 0.6209
Epoch 2/20
2627/2627 [==============================] - 3s 1ms/step - loss: 0.8935 - accuracy: 0.6221
Epoch 3/20
2627/2627 [==============================] - 3s 1ms/step - loss: 0.8933 - accuracy: 0.6220
Epoch 4/20
2627/2627 [==============================] - 3s 1ms/step - loss: 0.8937 - accuracy: 0.6216
Epoch 5/20
2627/2627 [==============================] - 3s 1ms/step - loss: 0.8927 - accuracy: 0.6224
Epoch 6/20
2627/2627 [==============================] - 3s 1ms/step - loss: 0.8927 - accuracy: 0.6226
Epoch 7/20
2627/2627 [==============================] - 3s 1ms/step - loss: 0.8928 - accuracy: 0.6227
Epoch 8/20
2627/2627 [==============================] - 3s 1ms/step - loss: 0.8927 - accuracy: 0.6231
Epoch 9/20
2627/2627 [==============================] - 3s 1ms/step - loss: 0.8923 - accuracy: 0.6228
Epoch 10/20
2627/2627 [==============================] - 3s 1ms/step - loss: 0.892

In [61]:
y_pred=model.predict(X_test)
y_test_class=np.argmax(y_test,axis=1)
y_pred_class=np.argmax(y_pred,axis=1)

In [62]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test_class,y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))

              precision    recall  f1-score   support

           0       0.55      0.14      0.22      6745
           1       0.00      0.00      0.00      1023
           2       0.63      0.96      0.76     13863
           3       0.00      0.00      0.00      1191

    accuracy                           0.62     22822
   macro avg       0.29      0.27      0.24     22822
weighted avg       0.54      0.62      0.52     22822

[[  915     0  5830     0]
 [   59     0   964     0]
 [  605     0 13258     0]
 [   97     0  1094     0]]


/Applications/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
